In [133]:
import numpy as np
import pandas as pd
from importer_snippets import load_experiment
from scipy.signal import find_peaks, savgol_filter
import matplotlib.pyplot as plt
plt.style.use('JGW')
import os
os.getcwd()

'C:\\Users\\jgage\\Documents\\GitHub\\CHEM274'

# Nyquist and Bode Plots

In [6]:
EIS = load_experiment()

In [ ]:
# Nyquist
fig = plt.figure()
ax = fig.add_subplot()

# ax.set_title('Nyquist Plot')
ax.tick_params(axis='both', which='both', direction='in', right=True, top=True)
# ax.set_prop_cycle(cc2)
ax.scatter(EIS.data['Zre'], -EIS.data['Zim'], label = 'data')
ax.set_xlabel('$Z_{re}$ / $\Omega$')
ax.set_ylabel('-$Z_{im}$ / $\Omega$')
ax.set_xlim ([0,1.1*max(EIS.data['Zre'])])
ax.set_ylim ([0,1.1*max(-EIS.data['Zim'])])
fig.tight_layout()

In [ ]:
# Bode
Z_mag = np.sqrt(EIS.data['Zre']**2 * EIS.data['Zim']**2)
log_Z_mag = np.log10(Z_mag)
phi = np.arctan(EIS.data['Zim'] / EIS.data['Zre']) * 180 / np.pi # phase angle in degrees
log_omega = np.log10(EIS.data['f'])


fig = plt.figure()
ax = fig.add_subplot()

# ax.set_title('Bode Plot')
ax.tick_params(axis='both', which='both', direction='in', right=True, top=True)
ax.scatter(log_omega, log_Z_mag, label = 'Impedance')
ax.set_ylabel('log$|Z|$', color='C0')
ax.set_xlabel('log$\omega$')

ax2 = ax.twinx()
ax2.scatter(log_omega, phi, label = 'Phase', color='C1')
ax2.set_ylabel('$\phi$ / degrees', color='C1')
plt.grid()
fig.tight_layout()

# Cyclic Voltammograms

In [144]:
CV = load_experiment()

In [ ]:
# CV
fig = plt.figure()
ax = fig.add_subplot()

# ax.set_title('Cyclic Voltammogram')
ax.tick_params(axis='both', which='both', direction='in', right=True, top=True)
# ax.set_prop_cycle(cc2)
ax.scatter(CV.data['Ecell'], CV.data['iw'], label = '$v=$ '+str(CV.params.loc[CV.params['parameter']=='scan_rate', 'value'].item()) + ' V/s')
ax.set_xlabel('$E_{cell}$ / V')
ax.set_ylabel('$i_{w}$ / $A$')

ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.style.use('JGW')
fig.legend()

### Find CV Peaks

In [ ]:
# Smooth data using the Savitski Golay method
smooth_iw = savgol_filter(CV.data['iw'], 25, 2)
# Check the result
# plt.plot(CV.data['Ecell'], smooth_iw, zorder=1, color='r')
# plt.plot(CV.data['Ecell'], CV.data['iw'], label = '$v=$ '+str(CV.params.loc[CV.params['parameter']=='scan_rate', 'value'].item()) + ' V/s', zorder=0)
# plt.xlim(-.75, -.74)
# plt.ylim(.00003, .00004)

# Maxima
find_ipa = find_peaks(smooth_iw, height=0, width=3600) # Must give minimum height to get peak_heights dict.
ipa, Epa = CV.data['iw'][find_ipa[0]].to_numpy(), CV.data['Ecell'][find_ipa[0]].to_numpy()

# Invert signal to find minima - Signals are negative but indicies are correct!
find_ipc = find_peaks(-smooth_iw, height=0, width=3600) # Remember to un-invert anytime if calling ipc values from find_ipc, but not indicies.
ipc, Epc = CV.data['iw'][find_ipc[0]].to_numpy(), CV.data['Ecell'][find_ipc[0]].to_numpy()

peaks = {'ipa': ipa, 'Epa': Epa, 'ipc': ipc, 'Epc': Epc}
peaks

#### Peak picking and figure saving

In [145]:
def save_CV_fig(fig, analyte, electrode, solvent, notebook):
    ans = input('Save the figure?')
    if ans in ['y', 'yes', 'Yes', 'yeye', 'yeah', 'why, certainly', True]:
        fig.savefig("C:\\Users\\jgage\\Documents\GitHub\\CHEM274\\data\\figs\\CV_"+
            analyte + electrode + solvent +
            str(CV.params.loc[CV.params['parameter']=='scan_rate', 'value'].item()) + 'Vs-1_'+
                notebook+'.jpg')
        print('Saved')
    else:
        print('Not saved')

In [ ]:
# Draw same plot as above, but with peaks
fig = plt.figure()
ax = fig.add_subplot()

ax.tick_params(axis='both', which='both', direction='in', right=True, top=True)
ax.plot(CV.data['Ecell'], CV.data['iw'], label = '$v=$ '+str(CV.params.loc[CV.params['parameter']=='scan_rate', 'value'].item()) + ' V/s', zorder=0)
ax.plot(CV.data['Ecell'], smooth_iw, color='C2', label='Savitsky-Golay', lw=.5, zorder=.5)
ax.scatter([*Epa, *Epc], [*ipa, *ipc], color='C1', label='Peaks', marker='|', s=200, zorder=1)
fig.tight_layout()

# ax.set_title('Cyclic Voltammogram')
ax.set_xlabel('$E_{cell}$ / V')
ax.set_ylabel('$i_{w}$ / $A$')
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
fig.legend(loc=(0.1, 0.78))

analyte = 'Feacac_'
electrode = 'GC_'
solvent = 'MeCN_'
notebook = '_P-03-05'

plt.show()
save_CV_fig(fig, analyte, electrode, solvent, notebook)